In [2]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict
import glob
import time

import pandas as pd
from pyarrow.lib import ArrowIOError
from tqdm import tqdm
import vaex

print(f"pd.__version__: {pd.__version__}")
print(f"vaex.__file__: {vaex.__file__}")
print(f"vaex.__version__: {vaex.__version__}")

pd.__version__: 0.25.3
vaex.__file__: /home/olga/miniconda3/envs/khtools--encodings-compare-cli--py37/lib/python3.7/site-packages/vaex/__init__.py
vaex.__version__: 1.0.0-beta.6


# Check disk-mounted storage

In [3]:
cd  ~/data_sm/kmer-hashing/quest-for-orthologs/analysis/2019/transcription-factors/intermediates

/mnt/data_sm/olga/kmer-hashing/quest-for-orthologs/analysis/2019/transcription-factors/intermediates


In [4]:
ls -1 | wc -l

26847


# Read orthologous TF info

In [5]:
data_folder = '/home/olga/data_sm/kmer-hashing/quest-for-orthologs/data/2019'
tfs = pd.read_parquet(f'{data_folder}/opisthokont_not_human_transcription_factors_ensembl_compara_merged_uniprot.parquet')
print(tfs.shape)
tfs.head()

(59232, 21)


,dn_ds,method_link_type,source__id,source__perc_id,source__perc_pos,source__protein_id,source__species,source__taxon_id,target__id,target__perc_id,...,target__protein_id,target__species,target__taxon_id,taxonomy_level,type,target__uniprot_id,target__id_type,target__db_id,target__species_id,target__taxa_id
0,NaN,ENSEMBL_PARALOGUES,ENSG00000267179,24.8792,37.1981,ENSP00000467286,homo_sapiens,9606,ENSG00000175787,17.0813,...,ENSP00000378792,homo_sapiens,9606,Bilateria,other_paralog,Q14929,Ensembl_PRO,ENSP00000378792,UP000005640,UP000005640
1,0.37910,ENSEMBL_PARALOGUES,ENSG00000181666,45.6753,57.5114,ENSP00000315505,homo_sapiens,9606,ENSG00000175787,49.9171,...,ENSP00000378792,homo_sapiens,9606,Bilateria,other_paralog,Q14929,Ensembl_PRO,ENSP00000378792,UP000005640,UP000005640
2,0.43907,ENSEMBL_PARALOGUES,ENSG00000164256,26.1745,34.0045,ENSP00000296682,homo_sapiens,9606,ENSG00000175787,38.8060,...,ENSP00000378792,homo_sapiens,9606,Bilateria,other_paralog,Q14929,Ensembl_PRO,ENSP00000378792,UP000005640,UP000005640
3,NaN,ENSEMBL_PARALOGUES,ENSG00000136866,30.5426,41.7054,ENSP00000452552,homo_sapiens,9606,ENSG00000175787,32.6700,...,ENSP00000378792,homo_sapiens,9606,Bilateria,other_paralog,Q14929,Ensembl_PRO,ENSP00000378792,UP000005640,UP000005640
4,NaN,ENSEMBL_PARALOGUES,ENSG00000187815,29.6578,41.4449,ENSP00000361791,homo_sapiens,9606,ENSG00000175787,25.8706,...,ENSP00000378792,homo_sapiens,9606,Bilateria,other_paralog,Q14929,Ensembl_PRO,ENSP00000378792,UP000005640,UP000005640


# Read parquet intermediates from local

## TODO: fix typo in "transcripton"

In [6]:
scratch_folder = '/home/olga/scratch/kmer-hashing/quest-for-orthologs/analysis/2019/transcripton-factors/intermediates'

In [7]:
arrow_folder = '/home/olga/scratch/kmer-hashing/quest-for-orthologs/analysis/2019/transcription-factors/intermediates-vaex-arrow'
! mkdir -p $arrow_folder

In [8]:
cd $scratch_folder

/home/olga/scratch/kmer-hashing/quest-for-orthologs/analysis/2019/transcripton-factors/intermediates


In [9]:
pwd

'/home/olga/scratch/kmer-hashing/quest-for-orthologs/analysis/2019/transcripton-factors/intermediates'

In [10]:
ls -1 | wc -l

17635


In [11]:
ls -lha | head

total 20G
drwxr-xr-x 2 olga czb  1.1M Jan 10 07:46 ./
drwxr-xr-x 4 olga czb  4.0K Jan 10 15:44 ../
-rw-r--r-- 1 olga czb     0 Jan 10 07:35 khtools_compare-kmers.log
-rw-r--r-- 1 olga czb  1.3M Jan 10 07:43 sp__A0JN51__ELF1_BOVIN.parquet
-rw-r--r-- 1 olga czb  1.5M Jan 10 07:45 sp__A0JNB1__ZN227_BOVIN.parquet
-rw-r--r-- 1 olga czb  1.4M Jan 10 07:43 sp__A0JPA0__CNNM4_XENTR.parquet
-rw-r--r-- 1 olga czb  1.2M Jan 10 07:39 sp__A0JPK3__ZFP57_RAT.parquet
-rw-r--r-- 1 olga czb  1.4M Jan 10 07:39 sp__A0JPL0__ZN382_RAT.parquet
-rw-r--r-- 1 olga czb  1.2M Jan 10 07:39 sp__A0JPN1__HLX_RAT.parquet
ls: write error


# Do just a subset for now --Read parquet files

In [12]:
%%time


def extract_protein_id(x):
    return x.split('|')[1]


ts = defaultdict(dict)
dfs = []
for i, parquet in zip(range(1000), tqdm(glob.iglob('*.parquet'))):
    prefix = parquet.split('.parquet')[0]
#     print(prefix)
    try:
        t0 = time.time()
        df = pd.read_parquet(parquet)
        t1 = time.time()
    except ArrowIOError:
        print(f'Could not read: {parquet}')
    ts[prefix]['pandas__read_parquet'] = t1-t0
#     print(ts[prefix])
    
#     t2 = time.time()
#     df['protein_id1'] = df.id1.map(extract_protein_id)
#     df['protein_id2'] = df.id2.map(extract_protein_id)
#     t3 = time.time()
#     ts[prefix]['extract_protein_id'] = t3-t2
    
#     # Merge with OMA output
#     t4 = time.time()
#     df_oma = df.merge(data_folder, how='outer', on=['protein_id1', 'protein_id2'])
#     t5 = time.time()
#     ts[prefix]['pandas__merge'] = t5-t4
# #     print(df_oma.shape)
#     df_oma.head()


    dfs.append(df)
#     break
    
times = pd.DataFrame(ts)
print(times.shape)
# print(f"times.median(): {times.median()}")
# print(f"times.mean(): {times.mean()}")
# print(f"times.std(): {times.std()}")
kmer_similarity = pd.concat(dfs)
print(kmer_similarity.shape)
kmer_similarity.head()

999it [07:57,  1.61it/s]

(1, 1000)
(1977656000, 5)
CPU times: user 8min 51s, sys: 9min 23s, total: 18min 15s
Wall time: 10min 35s


,id1,id2,ksize,jaccard,molecule
0,tr|A0A2I2Y780|A0A2I2Y780_GORGO Zinc finger pro...,tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized...,3,0.149701,protein
1,tr|A0A2I2Y780|A0A2I2Y780_GORGO Zinc finger pro...,tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized...,4,0.071038,protein
2,tr|A0A2I2Y780|A0A2I2Y780_GORGO Zinc finger pro...,tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized...,5,0.051680,protein
3,tr|A0A2I2Y780|A0A2I2Y780_GORGO Zinc finger pro...,tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized...,6,0.046683,protein
4,tr|A0A2I2Y780|A0A2I2Y780_GORGO Zinc finger pro...,tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized...,7,0.037825,protein


In [13]:
pwd

'/home/olga/scratch/kmer-hashing/quest-for-orthologs/analysis/2019/transcripton-factors/intermediates'

In [14]:
tf_folder = '/home/olga/scratch/kmer-hashing/quest-for-orthologs/analysis/2019/transcripton-factors'

In [15]:
%time kmer_similarity.to_parquet(f'{tf_folder}/kmer_similarity.parquet', index=False)

CPU times: user 12min 31s, sys: 6min 59s, total: 19min 31s
Wall time: 18min 52s


# Convert to Arrow

In [16]:
%%time
import vaex

kmer_similarity_vaex = vaex.from_pandas(kmer_similarity)
print(kmer_similarity_vaex.shape)
kmer_similarity_vaex.head()

(1977656000, 6)
CPU times: user 8h 19min 47s, sys: 1min 28s, total: 8h 21min 16s
Wall time: 8h 20min 52s


#,id1,id2,ksize,jaccard,molecule,index
0,'tr|A0A2I2Y780|A0A2I2Y780_GORGO Zinc finger prot...,'tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized ...,3,0.149701,protein,0
1,'tr|A0A2I2Y780|A0A2I2Y780_GORGO Zinc finger prot...,'tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized ...,4,0.0710383,protein,1
2,'tr|A0A2I2Y780|A0A2I2Y780_GORGO Zinc finger prot...,'tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized ...,5,0.0516796,protein,2
3,'tr|A0A2I2Y780|A0A2I2Y780_GORGO Zinc finger prot...,'tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized ...,6,0.046683,protein,3
4,'tr|A0A2I2Y780|A0A2I2Y780_GORGO Zinc finger prot...,'tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized ...,7,0.0378251,protein,4
5,'tr|A0A2I2Y780|A0A2I2Y780_GORGO Zinc finger prot...,'tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized ...,8,0.0322581,protein,5
6,'tr|A0A2I2Y780|A0A2I2Y780_GORGO Zinc finger prot...,'tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized ...,9,0.0293454,protein,6
7,'tr|A0A2I2Y780|A0A2I2Y780_GORGO Zinc finger prot...,'tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized ...,10,0.0244444,protein,7
8,'tr|A0A2I2Y780|A0A2I2Y780_GORGO Zinc finger prot...,'tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized ...,11,0.0197802,protein,8
9,'tr|A0A2I2Y780|A0A2I2Y780_GORGO Zinc finger prot...,'tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized ...,12,0.0152505,protein,9


In [17]:
kmer_similarity_vaex

#,id1,id2,ksize,jaccard,molecule,index
0,'tr|A0A2I2Y780|A0A2I2Y780_GORGO Zinc finger prot...,'tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized ...,3,0.1497005988023952,protein,0
1,'tr|A0A2I2Y780|A0A2I2Y780_GORGO Zinc finger prot...,'tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized ...,4,0.07103825136612021,protein,1
2,'tr|A0A2I2Y780|A0A2I2Y780_GORGO Zinc finger prot...,'tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized ...,5,0.05167958656330749,protein,2
3,'tr|A0A2I2Y780|A0A2I2Y780_GORGO Zinc finger prot...,'tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized ...,6,0.04668304668304668,protein,3
4,'tr|A0A2I2Y780|A0A2I2Y780_GORGO Zinc finger prot...,'tr|A0A087WUV0|A0A087WUV0_HUMAN Uncharacterized ...,7,0.037825059101654845,protein,4
...,...,...,...,...,...,...
"1,977,655,995",'sp|Q6P8Z1|MYCB_MOUSE Protein B-Myc OS=Mus muscu...,'tr|K7ELQ4|K7ELQ4_HUMAN Pro-FMRFamide-related ne...,41,0.0,hsdm17,339
"1,977,655,996",'sp|Q6P8Z1|MYCB_MOUSE Protein B-Myc OS=Mus muscu...,'tr|K7ELQ4|K7ELQ4_HUMAN Pro-FMRFamide-related ne...,42,0.0,hsdm17,340
"1,977,655,997",'sp|Q6P8Z1|MYCB_MOUSE Protein B-Myc OS=Mus muscu...,'tr|K7ELQ4|K7ELQ4_HUMAN Pro-FMRFamide-related ne...,43,0.0,hsdm17,341
"1,977,655,998",'sp|Q6P8Z1|MYCB_MOUSE Protein B-Myc OS=Mus muscu...,'tr|K7ELQ4|K7ELQ4_HUMAN Pro-FMRFamide-related ne...,44,0.0,hsdm17,342


## Write vaex files

In [ ]:
%%time
kmer_similarity_vaex.export_hdf5(f"{tf_folder}/kmer_similarity.hdf5")

In [ ]:
%%time
kmer_similarity_vaex.export_arrow(f"{tf_folder}/kmer_similarity.arrow")

# Read parquet files

In [ ]:
%%time


def extract_protein_id(x):
    return x.split('|')[1]


ts = defaultdict(dict)
dfs = []
for parquet in tqdm(glob.iglob('*.parquet')):
    prefix = parquet.split('.parquet')[0]
#     print(prefix)
    try:
        t0 = time.time()
        df = pd.read_parquet(parquet)
        t1 = time.time()
    except ArrowIOError:
        print(f'Could not read: {parquet}')
    ts[prefix]['pandas__read_parquet'] = t1-t0
#     print(ts[prefix])
    
#     t2 = time.time()
#     df['protein_id1'] = df.id1.map(extract_protein_id)
#     df['protein_id2'] = df.id2.map(extract_protein_id)
#     t3 = time.time()
#     ts[prefix]['extract_protein_id'] = t3-t2
    
#     # Merge with OMA output
#     t4 = time.time()
#     df_oma = df.merge(data_folder, how='outer', on=['protein_id1', 'protein_id2'])
#     t5 = time.time()
#     ts[prefix]['pandas__merge'] = t5-t4
# #     print(df_oma.shape)
#     df_oma.head()


    dfs.append(df)
#     break
    
times = pd.DataFrame(ts)
print(times.shape)
print(f"times.median(): {times.median()}")
print(f"times.mean(): {times.mean()}")
print(f"times.std(): {times.std()}")
kmer_similarity = pd.concat(dfs)
print(kmer_similarity.shape)
kmer_similarity.head()

## stop here

In [ ]:
assert False

# Playing around with stuf

In [ ]:
    t2 = time.time()
    df['protein_id1'] = df.id1.map(extract_protein_id)
    df['protein_id2'] = df.id2.map(extract_protein_id)
    t3 = time.time()
    ts[prefix]['extract_protein_id'] = t3-t2

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df_merged = df.merge(tfs, left_on='protein_id1', right_on='uniprot_id')
print(df_merged.shape)

In [ ]:
df_merged.head()

In [ ]:
df_merged.tail()

In [ ]:
df.merge()

# Convert to arrow

In [ ]:
%%time


def extract_protein_id(x):
    return x.split('|')[1]


ts = defaultdict(dict)
dfs = []
for parquet in glob.iglob('*.parquet'):
    prefix = parquet.split('.parquet')[0]
    print(prefix)
    try:
        t0 = time.time()
        df = pd.read_parquet(parquet)
        t1 = time.time()
    except ArrowIOError:
        print(f'Could not read: {parquet}')
    ts[prefix]['pandas__read_parquet'] = t1-t0
    print(ts[prefix])
    
#     t2 = time.time()
#     df['protein_id1'] = df.id1.map(extract_protein_id)
#     df['protein_id2'] = df.id2.map(extract_protein_id)
#     t3 = time.time()
#     ts[prefix]['extract_protein_id'] = t3-t2
    
#     # Merge with OMA output
#     t4 = time.time()
#     df_oma = df.merge(oma, how='outer', on=['protein_id1', 'protein_id2'])
#     t5 = time.time()
#     ts[prefix]['pandas__merge'] = t5-t4
# #     print(df_oma.shape)
# #     df_oma.head()

    t6 = time.time()
    df_vx = vaex.from_pandas(df)
    t7 = time.time()
    ts[prefix]['vaex__from_pandas'] = t7 - t6
    print(ts[prefix])
    
    arrow = f'{arrow_folder}/{prefix}.arrow'

    t8 = time.time()
    df_vx.export_arrow(arrow, progress=True)
    t9 = time.time()
    ts[prefix]['vaex__export_arrow'] = t9 - t8
    print(ts[prefix])
#     del df
#     del df_vx
#     dfs.append(df)
    break
    
times = pd.DataFrame(ts)
print(times.shape)
print(f"times.median(): {times.median()}")
print(f"times.mean(): {times.mean()}")
print(f"times.std(): {times.std()}")
# kmer_similarity = pd.concat(dfs)
# print(kmer_similarity.shape)
# kmer_similarity.head()

In [ ]:
df_vx.head()

In [ ]:
prefix

## Try exporting to different formats

In [ ]:
hdf5 = f'/home/olga/scratch/{prefix}.hdf5'
%time df_vx.export_hdf5(hdf5)

In [ ]:
! ls -lha $hdf5

In [ ]:
%time from_hdf5 = vaex.open(hdf5)
from_hdf5.head()

In [ ]:
arrow = f'/home/olga/scratch/{prefix}.arrow'
%time df_vx.export_arrow(arrow)

! ls -lha $arrow

In [ ]:
%time from_arrow = vaex.open(arrow)
from_arrow.head()

In [ ]:
500 * 2000 / 60 / 60

277 hours???

In [ ]:
ts

In [ ]:
df.shape

In [ ]:
len(dfs)

In [ ]:
# kmer_similarity = pd.concat(dfs)

In [ ]:
print(kmer_similarity.shape)
kmer_similarity.head()

In [ ]:
f'{len(kmer_similarity.index):,}'

# Convert to vaex for speed

In [ ]:
import vaex as vx

In [ ]:
parquet

In [ ]:
vaex.from_arrow_table(parquet)

# Clean data

## Extract protein ids from full name

### Get unique ids as making a map on unique ids is faster

In [ ]:
%%time
%time id1 = kmer_similarity.id1.unique()
%time id2 = kmer_similarity.id2.unique()


In [ ]:
all_ids = set(id1) + set(id2)
len(all_ids)

In [ ]:
%%time


id1_proteins = {x: extract_protein_id(x) for x in id1}
id2_proteins = {x: extract_protein_id(x) for x in id2}

kmer_similarity['protein_id1'] = kmer_similarity.id1.map(extract_protein_id)
kmer_similarity['protein_id2'] = kmer_similarity.id2.map(extract_protein_id)
# kmer_similarity.head()

## How many of different proteins are there

In [ ]:
kmer_similarity.protein_id1.nunique()

In [ ]:
kmer_similarity.protein_id2.nunique()

## Check where jaccard is greater than 0

In [ ]:
(kmer_similarity.jaccard > 0).sum()

In [ ]:
kmer_similarity.query('jaccard > 0.8')

In [ ]:
(kmer_similarity.jaccard > 0.8).sum()